# Carregar os dados

In [14]:
#%pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 228.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 146.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
#!kaggle datasets download -d shanegerami/ai-vs-human-text

Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text
License(s): other
 96%|███████████████████████████████████████▌ | 337M/350M [00:02<00:00, 137MB/s]
100%|█████████████████████████████████████████| 350M/350M [00:03<00:00, 122MB/s]


In [3]:
#import zipfile

# Substitua o nome do arquivo pelo nome correto
#with zipfile.ZipFile("ai-vs-human-text.zip", 'r') as zip_ref:
#    zip_ref.extractall("dados_textos")  # Pasta onde os dados serão extraídos


In [1]:
caminho = '/teamspace/studios/this_studio/dados_textos/AI_Human.csv'

In [2]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel,BertTokenizer
from sklearn.model_selection import train_test_split

In [3]:
# Carregar o dataset
df = pd.read_csv(caminho)
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [4]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['text'].values, 
    df['generated'].values,
    test_size=0.2,
    random_state=42
)

In [5]:
# Inicializar o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Criando uma classe personalizada para o dataset

In [6]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }


Essa classe **`TextClassificationDataset`** é uma implementação personalizada do `Dataset` do PyTorch para **classificação de textos** usando um `tokenizer`, como os da biblioteca `transformers` (BERT, DistilBERT, etc.).  

Vamos analisar **passo a passo** o que ela faz. 🚀  

---

### **📌 O que essa classe faz?**
Ela **recebe** um conjunto de textos e rótulos, **aplica tokenização** e **retorna tensores formatados** para serem usados no treinamento de um modelo.

### **📌 Estrutura e funcionamento**

#### 🔹 **1. Construtor `__init__`**
```python
def __init__(self, texts, labels, tokenizer, max_length=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length
```
👉 **O que acontece aqui?**  
- **`texts`** → Lista de textos brutos.  
- **`labels`** → Lista de rótulos correspondentes.  
- **`tokenizer`** → Tokenizador, geralmente um modelo pré-treinado do Hugging Face, como BERT.  
- **`max_length=512`** → Define o comprimento máximo da sequência (o BERT suporta até 512 tokens).  

---

#### 🔹 **2. Método `__len__`**
```python
def __len__(self):
    return len(self.texts)
```
👉 **Retorna o número de amostras** no dataset. Isso é necessário para que o `DataLoader` saiba quantos exemplos existem.

---

#### 🔹 **3. Método `__getitem__`**
```python
def __getitem__(self, idx):
    text = str(self.texts[idx])  # Converte o texto para string
    label = self.labels[idx]  # Obtém o rótulo correspondente
```
👉 **Pega um texto e seu rótulo correspondente pelo índice `idx`**.

---

#### 🔹 **4. Tokenização com `tokenizer.encode_plus`**
```python
encoding = self.tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=self.max_length,
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
```
👉 **Aqui acontece a mágica!** Ele **converte o texto em números** para que a rede neural consiga processá-lo. Vamos entender cada parâmetro:

| Parâmetro | O que faz? |
|-----------|-----------|
| `text` | Texto de entrada. |
| `add_special_tokens=True` | Adiciona tokens `[CLS]` e `[SEP]` (essenciais para modelos como BERT). |
| `max_length=self.max_length` | Define o tamanho máximo da sequência. |
| `return_token_type_ids=False` | Não retorna token types (usado para tarefas como QA, não para classificação). |
| `padding='max_length'` | Garante que todas as sequências tenham o mesmo tamanho, preenchendo com `[PAD]` se necessário. |
| `truncation=True` | Se o texto for maior que `max_length`, ele será cortado. |
| `return_attention_mask=True` | Retorna uma máscara indicando quais tokens são reais (1) e quais são padding (0). |
| `return_tensors='pt'` | Retorna tensores do PyTorch (`torch.Tensor`). |

---

#### 🔹 **5. Retorno final**
```python
return {
    'input_ids': encoding['input_ids'].flatten(),
    'attention_mask': encoding['attention_mask'].flatten(),
    'labels': torch.tensor(label, dtype=torch.float)
}
```
👉 **Esse dicionário é o que o `DataLoader` recebe e retorna durante o treinamento**:

- **`input_ids`** → Os tokens do texto convertidos em números.  
- **`attention_mask`** → Indica quais tokens são válidos (1) e quais são padding (0).  
- **`labels`** → O rótulo da amostra como um tensor do PyTorch.  

---

### **📌 Exemplo de Uso**
Agora vamos ver como essa classe é usada na prática:

```python
from transformers import BertTokenizer

# Carregar o tokenizador do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Criar listas de textos e rótulos
texts = ["This is an AI-generated text.", "Humans write differently than AI."]
labels = [1, 0]

# Criar o dataset
dataset = TextClassificationDataset(texts, labels, tokenizer)

# Testar a saída
sample = dataset[0]
print(sample)
```

#### **📌 Saída esperada**
```python
{
    'input_ids': tensor([101, 2023, 2003, 2019, ...]),
    'attention_mask': tensor([1, 1, 1, ...]),
    'labels': tensor(1.)
}
```

---

### **📌 Resumo**
Essa classe **transforma um conjunto de textos e rótulos em tensores prontos para serem usados no PyTorch**, aplicando:
✅ **Tokenização automática** com modelos da Hugging Face.  
✅ **Padding/truncamento automático** para manter tamanhos uniformes.  
✅ **Conversão para tensores do PyTorch** para facilitar o treinamento.

Se precisar de mais detalhes ou quiser treinar um modelo com essa abordagem, me avise! 🚀🔥

In [7]:
# Criar datasets
train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
test_dataset = TextClassificationDataset(X_test, y_test, tokenizer)

In [8]:
# Criar dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Criando o modelo

In [9]:
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]
        dropout_output = self.dropout(pooled_output)
        logits = self.fc(dropout_output)
        return self.sigmoid(logits)


Esse código define uma rede neural chamada `TextClassifier` que usa **BERT** para classificar textos. Vou explicar **passo a passo** o que acontece. 🚀  

---

## **📌 O que esse modelo faz?**  
- Usa **BERT** como base para extrair representações dos textos.  
- Passa a saída do BERT por uma **camada totalmente conectada (`fc`)** para classificação.  
- Usa **sigmoide (`sigmoid`)** para produzir uma saída entre `0` e `1`, útil para **classificação binária** (IA vs. humano, por exemplo).  

---

## **📌 Estrutura e funcionamento**
Vamos destrinchar o código.

### **1️⃣ Construtor `__init__`**
```python
def __init__(self):
    super(TextClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.dropout = nn.Dropout(0.1)
    self.fc = nn.Linear(768, 1)
    self.sigmoid = nn.Sigmoid()
```

👉 **O que acontece aqui?**  
1. **`self.bert = BertModel.from_pretrained('bert-base-uncased')`**  
   - Carrega o modelo **BERT pré-treinado** (versão `bert-base-uncased`, que não diferencia maiúsculas/minúsculas).  
   - BERT gera representações dos textos em um espaço vetorial de **768 dimensões** (para `bert-base`).  

2. **`self.dropout = nn.Dropout(0.1)`**  
   - Aplica **Dropout** com `10%` de taxa para evitar overfitting.  

3. **`self.fc = nn.Linear(768, 1)`**  
   - Camada **totalmente conectada (`Linear`)** que reduz as 768 dimensões do BERT para **1 único valor**, que será a **probabilidade da classe 1** (texto gerado por IA).  

4. **`self.sigmoid = nn.Sigmoid()`**  
   - Aplica a **função sigmoide** para converter o valor final em uma **probabilidade entre 0 e 1**.  

---

### **2️⃣ Método `forward` (Propagação para Frente)**
```python
def forward(self, input_ids, attention_mask):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    pooled_output = outputs[1]
    dropout_output = self.dropout(pooled_output)
    logits = self.fc(dropout_output)
    return self.sigmoid(logits)
```

👉 **Explicação passo a passo**:
1. **Executa o BERT**
   ```python
   outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
   ```
   - O `input_ids` representa os tokens dos textos.
   - O `attention_mask` diz quais tokens são reais (`1`) e quais são padding (`0`).
   - O BERT retorna **duas saídas principais**:
     1. `outputs[0]` → Embeddings de cada palavra (não usados aqui).
     2. `outputs[1]` → Representação do token `[CLS]`, que é um **resumo do significado do texto**.  

2. **Extrai o vetor do `[CLS]`**
   ```python
   pooled_output = outputs[1]
   ```
   - O primeiro token de cada texto no BERT é sempre o **token especial `[CLS]`**, que contém uma **representação global** da sentença.  

3. **Aplica Dropout**
   ```python
   dropout_output = self.dropout(pooled_output)
   ```
   - O Dropout ajuda a evitar overfitting removendo aleatoriamente alguns valores durante o treinamento.  

4. **Passa pela camada totalmente conectada**
   ```python
   logits = self.fc(dropout_output)
   ```
   - A camada `fc` reduz o vetor de **768 dimensões** para **1 única saída**, que representa a **probabilidade do texto ser IA**.  

5. **Aplica a sigmoide**
   ```python
   return self.sigmoid(logits)
   ```
   - Converte o valor da `fc` para um número entre **0 e 1**, que será interpretado como **probabilidade da classe 1**.  
   - Se for maior que `0.5`, pode ser considerado "Texto gerado por IA", senão "Texto humano".  



# Configurar o treinamento

In [10]:
# Inicializar modelo
model = TextClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Definir otimizador e função de perda
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()

# Número de épocas
num_epochs = 3


# Loop de treinamento

In [11]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f'Época {epoch+1}/{num_epochs}, Perda média: {avg_loss:.4f}')


Época 1/3, Perda média: 0.0121


# Avaliação dos resultados

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def evaluate_model(model, test_loader, device):
    model.eval()  # Coloca o modelo em modo de avaliação
    predictions = []
    actual_labels = []
    
    with torch.no_grad():  # Desativa o cálculo de gradientes
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Fazer predições
            outputs = model(input_ids, attention_mask)
            # Converter as probabilidades em classes (0 ou 1)
            preds = (outputs.squeeze() >= 0.5).float()
            
            # Mover para CPU e converter para lista
            predictions.extend(preds.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())
    
    # Converter listas para arrays numpy
    predictions = np.array(predictions)
    actual_labels = np.array(actual_labels)
    
    # Calcular métricas
    accuracy = accuracy_score(actual_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        actual_labels, 
        predictions, 
        average='binary'
    )
    
    # Imprimir resultados
    print("\nResultados da Avaliação:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Chamar a função de avaliação
metrics = evaluate_model(model, test_loader, device)

# Opcional: Salvar o modelo treinado
torch.save(model.state_dict(), 'modelo_classificador.pt')

# Para carregar o modelo posteriormente:
# model.load_state_dict(torch.load('modelo_classificador.pt'))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusão')
    plt.ylabel('Valor Real')
    plt.xlabel('Valor Previsto')
    plt.show()

# Para usar a matriz de confusão, adicione ao final da função evaluate_model:
plot_confusion_matrix(actual_labels, predictions)


# Avaliação em um único texto

In [ ]:
def predict_single_text(text, model, tokenizer, device):
    # Coloca o modelo em modo de avaliação
    model.eval()
    
    # Tokeniza o texto
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move para o device apropriado (CPU ou GPU)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Faz a predição
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probability = outputs.squeeze().item()
        prediction = 1 if probability >= 0.5 else 0
    
    # Retorna os resultados
    return {
        'texto': text,
        'probabilidade': probability,
        'previsao': 'IA' if prediction == 1 else 'Humano',
    }

# Exemplo de uso:
texto = "Olá, bom dia! Como você está hoje?"
resultado = predict_single_text(texto, model, tokenizer, device)

print("\nResultados da Análise:")
print(f"Texto: {resultado['texto']}")
print(f"Probabilidade de ser IA: {resultado['probabilidade']:.4f}")
print(f"Classificação: {resultado['previsao']}")


In [ ]:
# Testando com diferentes textos
textos_teste = [
    "Olá, bom dia! Como você está hoje?",
    "O processo de fotossíntese é um mecanismo biológico complexo que envolve a conversão de energia luminosa em energia química.",
    "Cara, nem acredito que perdi o ônibus hoje de manhã! Que azar...",
]

for texto in textos_teste:
    resultado = predict_single_text(texto, model, tokenizer, device)
    print("\n" + "="*50)
    print(f"Texto: {resultado['texto']}")
    print(f"Probabilidade de ser IA: {resultado['probabilidade']:.4f}")
    print(f"Classificação: {resultado['previsao']}")


Lembre-se que você precisa usar o mesmo tokenizer que foi usado no treinamento do modelo. Se você estiver carregando um modelo salvo, certifique-se de também ter o tokenizer correto:

In [ ]:
# Se você precisar carregar o modelo e tokenizer novamente:
from transformers import BertTokenizer

# Carregar o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Carregar o modelo (assumindo que você já tem a classe do modelo definida)
model = TextClassifier()  # Substitua pelo nome da sua classe de modelo
model.load_state_dict(torch.load('modelo_classificador.pt'))
model.to(device)
